In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <complex.h>
#include <math.h>
#include <string.h>

#define PI 3.14159265358979323846

// Bit reversal for reordering
unsigned int bit_reverse(unsigned int x, int log2n) {
unsigned int n = 0;
for (int i = 0; i < log2n; i++) {
n <<= 1;
n |= (x & 1);
x >>= 1;
}
return n;
}

// Print complex array
void print_complex_array(double complex *x, int N, const char *label) {
printf("%s:\n", label);
for (int i = 0; i < N; i++) {
printf("x[%d] = %.4f + %.4fj\n", i, creal(x[i]), cimag(x[i]));
}
printf("\n");
}

// Radix-2 DIT FFT
void fft_dit(double complex *x, int N) {
int log2N = log2(N);
for (int i = 0; i < N; i++) {
int r = bit_reverse(i, log2N);
if (i < r) {
double complex temp = x[i];
x[i] = x[r];
x[r] = temp;
}
}

for (int s = 1; s <= log2N; s++) {
int m = 1 << s;
double complex wm = cexp(-2.0 * PI * I / m);
for (int k = 0; k < N; k += m) {
double complex w = 1.0;
for (int j = 0; j < m / 2; j++) {
double complex t = w * x[k + j + m / 2];
double complex u = x[k + j];
x[k + j] = u + t;
x[k + j + m / 2] = u - t;
w *= wm;
}
}
char label[32];
sprintf(label, "FFT Stage %d Output", s);
print_complex_array(x, N, label);
}
}

// Radix-2 DIF FFT
void fft_dif(double complex *x, int N) {
int log2N = log2(N);

for (int s = log2N; s >= 1; s--) {
int m = 1 << s;
double complex wm = cexp(-2.0 * PI * I / m);
for (int k = 0; k < N; k += m) {
double complex w = 1.0;
for (int j = 0; j < m / 2; j++) {
double complex u = x[k + j];
double complex v = x[k + j + m / 2];
x[k + j] = u + v;
x[k + j + m / 2] = (u - v) * w;
w *= wm;
}
}
char label[32];
sprintf(label, "FFT Stage %d Output", log2N - s + 1);
print_complex_array(x, N, label);
}

for (int i = 0; i < N; i++) {
int r = bit_reverse(i, log2N);
if (i < r) {
double complex temp = x[i];
x[i] = x[r];
x[r] = temp;
}
}
}

// Radix-2 DIT IFFT
void ifft_dit(double complex *x, int N) {
int log2N = log2(N);

// Bit-reversal reordering
for (int i = 0; i < N; i++) {
int r = bit_reverse(i, log2N);
if (i < r) {
double complex temp = x[i];
x[i] = x[r];
x[r] = temp;
}
}

// Inverse FFT computation using positive angle
for (int s = 1; s <= log2N; s++) {
int m = 1 << s;
double complex wm = cexp(+2.0 * PI * I / m); // + angle for IFFT
for (int k = 0; k < N; k += m) {
double complex w = 1.0;
for (int j = 0; j < m / 2; j++) {
double complex t = w * x[k + j + m / 2];
double complex u = x[k + j];
x[k + j] = u + t;
x[k + j + m / 2] = u - t;
w *= wm;
}
}
char label[32];
sprintf(label, "IFFT Stage %d Output", s);
print_complex_array(x, N, label);
}

// Divide by N
for (int i = 0; i < N; i++) {
x[i] /= N;
}
}

int main() {
int N;
printf("Enter N (power of 2): ");
scanf("%d", &N);

if ((N & (N - 1)) != 0) {
printf("Error: N must be a power of 2.\n");
return 1;
}

double complex *x = malloc(N * sizeof(double complex));
double complex *x_out = malloc(N * sizeof(double complex));
double complex *temp = malloc(N * sizeof(double complex));

int operation;
printf("Choose Operation:\n1. FFT only\n2. IFFT only\n3. FFT then IFFT\nEnter choice: ");
scanf("%d", &operation);

if (operation == 1 || operation == 3) {
int fft_type;
printf("Choose FFT Type:\n1. DIT FFT\n2. DIF FFT\nEnter choice: ");
scanf("%d", &fft_type);

printf("Enter real and imaginary parts of input signal:\n");
for (int i = 0; i < N; i++) {
double real, imag;
printf("x[%d] = ", i);
scanf("%lf %lf", &real, &imag);
x[i] = real + imag * I;
}

memcpy(temp, x, N * sizeof(double complex));

if (fft_type == 1) {
fft_dit(temp, N);
} else if (fft_type == 2) {
fft_dif(temp, N);
} else {
printf("Invalid FFT type.\n");
goto cleanup;
}

printf("FFT Output:\n");
for (int i = 0; i < N; i++) {
printf("X[%d] = %.4f + %.4fj\n", i, creal(temp[i]), cimag(temp[i]));
}

if (operation == 1) goto cleanup;

memcpy(x, temp, N * sizeof(double complex)); // Feed FFT output to IFFT
}

if (operation == 2) {
printf("Enter DFT values (real and imaginary):\n");
for (int i = 0; i < N; i++) {
double real, imag;
printf("X[%d] = ", i);
scanf("%lf %lf", &real, &imag);
x[i] = real + imag * I;
}
}

memcpy(x_out, x, N * sizeof(double complex));
ifft_dit(x_out, N);

printf("IFFT Output:\n");
for (int i = 0; i < N; i++) {
printf("x[%d] = %.4f + %.4fj\n", i, creal(x_out[i]), cimag(x_out[i]));
}

cleanup:
free(x);
free(x_out);
free(temp);
return 0;
}